In [1]:
import os
import cv2
import imutils
import numpy as np
import cvlib as cv
import matplotlib.pyplot as plt
from cvlib.object_detection import draw_bbox

In [2]:
# This fuction remove the components that are smaller than praticular threshold
def f_keepLargeComponents(I,th):
    R = np.zeros(I.shape)<0
    unique_labels = np.unique(I.flatten())
    for label in unique_labels:
        if label == 0:
            pass
        else:
            I2 = I==label
            if np.sum(I2)>th:
                R = R | I2
    return np.float32(255*R)

In [3]:
def f_saveSeq(C,counter,th,outPutPath):
    if len(C)<th:
        pass
    else:
        k = 1
        for frame in C:
            imName = str(counter)+'_'+ str(k)+'.jpg'
            finalPath = os.path.join(outPutPath,imName)
            bbox,labels,conf = cv.detect_common_objects(frame)
            frame = draw_bbox(frame,bbox,labels,conf)
            cv2.imwrite(finalPath,frame)
            k += 1

In [4]:
def f_displaySeq(outPutPath):
    for imName in os.listdir(outPutPath):
        frame = cv2.imread(os.path.join(outPutPath,imName))
        frame = cv2.resize(frame,dsize=(600,400))
        cv2.imshow('Display',frame)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    cv2.destroyAllWindows()

In [5]:
cap = cv2.VideoCapture(1) # The device number might be 0 or 1 depending on the device and the webcam

In [ ]:
cap.open(0, cv2.CAP_DSHOW)
fgModel = cv2.createBackgroundSubtractorMOG2()
leastNumOfFrames = 5
idx = []
C = []
counter = 0
outPutPath = (r'C:\Users') # add folder location to save
while(True):
    ret, frame = cap.read()
    if not ret:
        break
        
    counter += 1
    frame = cv2.resize(frame,dsize=(500,300))
    fgmask = fgModel.apply(frame)
    K_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,K_r)
    num_labels,labels_im = cv2.connectedComponents(np.array(fgmask>0,np.uint8))
    fgmask = f_keepLargeComponents(labels_im,1000)
    if np.sum(fgmask)>0:
        idx.append(counter)
        C.append(frame)
    if len(idx) >= 2 and idx[-1] > idx[-2]+1:
        f_saveSeq(C,counter,leastNumOfFrames,outPutPath)
        idx = []
        C = []
        
    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask,fgmask
    F2 = np.hstack((frame,F))
    cv2.imshow('frame', frame)

    #add the code here

    if cv2.waitKey(1) & 0xFF == ord('q'): #to quit the stream
        break
f_saveSeq(C,counter,leastNumOfFrames,outPutPath)
cap.release()
cv2.destroyAllWindows()

# Show the detection

In [1]:
imPath = (r'C:\Users') # add file(video) location 

In [ ]:
for imName in os.listdir(imPath):
    frame = cv2.imread(os.path.join(imPath,imName))
    frame = cv2.resize(frame,dsize=(600,400))
    cv2.imshow('Display',frame)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cv2.destroyAllWindows()